In [17]:
import requests
from scipy.stats import gamma
import numpy as np
import orjson as json
import time
import altair as alt
import pandas as pd
# alt.renderers.enable('notebook')


VM_URL="http://localhost:8428/api/v1/import"


def to_metric_json(name, labels, timestamps, values):
    return json.dumps({"metric": {"__name__": name, **labels},
                       "values": values,
                       "timestamps": timestamps
                       })


def send_metric(url, name, labels, timestamps, values):
    resp = requests.post(url, data=to_metric_json(name, labels, timestamps, values))
    print(f"HTTP {resp.status_code}")


def gen_timestamps(n, resolution, end=None):
    if end is None:
        end = time.time()
    return np.linspace(end - n, end, int(n/resolution), dtype=np.int64)*1000


In [43]:
# Delete metrics
def delete_by_query(query):
    resp = requests.get("http://localhost:8428/api/v1/admin/tsdb/delete_series", params={"match[]": query})
    if not resp.ok:
        print(resp.text)
    else:
        print("Deleted")
        
# delete_by_query('{__name__!=""}')

Deleted


In [18]:
# base signal, high res
n=3600
resolution=1 # sec
noise = gamma.rvs(2.2, size=n)
signal = np.sin(np.linspace(0, np.pi*3, n))*2 + noise
signal[1023] = 30
signal[2034] = 45
timestamps = gen_timestamps(n, 1)

original_gauge = alt.Chart(pd.DataFrame({"time": timestamps, "Y": signal})).mark_line(size=1)\
    .encode(x="time:T", y="Y").properties(width="container")
original_gauge

alt.Chart(...)

In [61]:
import random
window_size = 60
lowres = np.split(signal, signal.shape[0]/window_size)
lowres_timestamps = timestamps[::window_size]
lowres_sample = [random.choice(r) for r in lowres]
lowres_avg = [r.mean() for r in lowres]
lowres_max = [r.max() for r in lowres]

c_base = alt.Chart(pd.DataFrame({"time": lowres_timestamps, "mean": lowres_avg, "gauge": lowres_sample, "max": lowres_max})).encode(x="time:T")
alt.layer(c_base.mark_line(color="blue").encode(y="gauge"),
          c_base.mark_line(color="green").encode(y="mean"),
          c_base.mark_line(color="red").encode(y="max"),
          original_gauge
          ).properties(width="container")

alt.LayerChart(...)

In [42]:
# %%script echo skipping 
# base signal, high res
# delete_by_query('{_s_name__=~"demo_low_res.*"}')

n=24*3600
resolution=1 # sec
noise = gamma.rvs(2.2, size=n)
signal = np.sin(np.linspace(0, np.pi*3, n))*2 + noise
peaks_idx = np.random.choice(np.linspace(0, signal.shape[0], signal.shape[0], dtype="int64"), size=10)
signal[peaks_idx] = [30 + np.random.randint(5, 20)]
now = time.time()
timestamps = np.linspace(now - n*resolution, now, n, dtype=np.int64)*1000
lowres_window = 10
signal[:-1], signal[1:]
counter = np.cumsum(signal)[::lowres_window]
lowres_timestamps = timestamps[::lowres_window]
lowres = np.split(signal, signal.shape[0]/lowres_window)
lowres_sample = [float(np.random.choice(r)) for r in lowres]
lowres_avg = [float(r.mean()) for r in lowres]

send_metric(VM_URL, "demo_high_res_gauge", {"host": "demo", "res": "1sec"}, timestamps.tolist(), signal.tolist())
send_metric(VM_URL, "demo_low_res_counter", {"host": "demo", "res": f"{lowres_window}sec"}, lowres_timestamps.tolist(), counter.tolist())
send_metric(VM_URL, "demo_low_res_gauge", {"host": "demo", "res": f"{lowres_window}s"}, lowres_timestamps.tolist(), lowres_sample)
send_metric(VM_URL, "demo_low_res_gauge_mean", {"host": "demo", "res": f"{lowres_window}s"}, lowres_timestamps.tolist(), lowres_avg)



Deleted
HTTP 204
HTTP 204
HTTP 204
HTTP 204


In [27]:
n=24*3600
window = 30
points = int(n/window)
timestamps = gen_timestamps(n, window)
signal = (np.sin(np.linspace(0, np.pi*24, points))+1)*2 + gamma.rvs(3, size=points)*0.5
issue_started = np.random.randint(0, int(points/2))
signal[issue_started:issue_started+20] += np.sin(np.linspace(0, np.pi/2, 20))*10

# alt.Chart(pd.DataFrame({"time": timestamps, "signal": signal}))\
#     .mark_line().encode(x="time:T", y="signal")\
#     .properties(width="container")
    
send_metric(VM_URL, "demo_zero_bound_with_issue", {"host": "demo", "res": "30s"}, timestamps.tolist(), signal.tolist())


HTTP 204


In [37]:
# metric with large peak overshadowing signal (by triggering auto axis bounds)
# delete_by_query("demo_large_peak_shadow")

n=24*3600
window = 30
points = int(n/window)
timestamps = gen_timestamps(n, window)
signal = (np.sin(np.linspace(0, np.pi*24, points))+1)*2 + gamma.rvs(3, size=points)*0.5
issue_started = np.random.randint(0, int(points/2))
signal[issue_started:issue_started+10] += np.sin(np.linspace(0, np.pi/2, 10))*1000



# alt.Chart(pd.DataFrame({"time": timestamps, "signal": signal}))\
#     .mark_line().encode(x="time:T", y="signal")\
#     .properties(width="container")
    
send_metric(VM_URL, "demo_large_peak_shadow", {"host": "demo", "res": "30s"}, timestamps.tolist(), signal.tolist())


Deleted
HTTP 204


In [31]:
# metric with holes (missing datapoints) masked by line interpolation

n=24*3600
window = 30
points = int(n/window)
timestamps = gen_timestamps(n, window)
signal = (np.sin(np.linspace(0, np.pi*24, points))+1)*2 + gamma.rvs(3, size=points)*0.5
issues = np.random.randint(0, points, size=20)
for issue in issues:
    signal[issue:issue+5] = np.NaN


# alt.Chart(pd.DataFrame({"time": timestamps, "signal": signal}))\
#     .mark_line().encode(x="time:T", y="signal")\
#     .properties(width="container")
    
send_metric(VM_URL, "demo_missing_points", {"host": "demo", "res": "30s"}, timestamps.tolist(), signal.tolist())


Deleted
HTTP 204
